### Multipleclass Classification

In [55]:
# 필요한 라이브러리 호출
import torch
from torch.optim import Adam
from torch.nn import Linear, MSELoss, Sequential, Module, Softmax, CrossEntropyLoss
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

In [3]:
# 랜덤 데이터 생성 (2행 3열)
r = torch.randn(2, 3)
r

tensor([[ 0.2147, -1.4382,  1.3584],
        [-0.3699, -0.5611, -0.1840]])

In [5]:
# Softmax 함수 사용
sfx = Softmax(dim=1) # 행별 확률값
sfx(r) # sfx.forward()

tensor([[0.2310, 0.0442, 0.7248],
        [0.3300, 0.2726, 0.3974]])

In [18]:
# 데이터 불러오기
df = pd.read_csv('data/softmax.txt', sep=' ', header=None)
df

,0,1,2,3,4
0,2,1,0,0,1
1,3,2,0,0,1
2,3,4,0,0,1
3,5,5,0,1,0
4,7,5,0,1,0
5,2,5,0,1,0
6,6,6,1,0,0
7,7,7,1,0,0


In [23]:
# 행(feature), 열(label) 추출
x_data = df.iloc[:, :2].values
y_data = df.iloc[:, 2:].values

In [24]:
# col별 가장 큰 데이터의 인덱스 추출
y_data = y_data.argmax(axis=1)
y_data

array([2, 2, 2, 1, 1, 1, 0, 0], dtype=int64)

In [31]:
# x, y, w, b 준비
x = torch.FloatTensor(x_data)
y = torch.LongTensor(y_data)
w = torch.empty([2, 3], requires_grad=True) # feature : 2개, label : 3개
b = torch.empty([3], requires_grad=True) # label : 3개
torch.nn.init.uniform_(w)
torch.nn.init.uniform_(b)

tensor([0.5221, 0.0792, 0.6855], requires_grad=True)

In [32]:
# cost function 정의
def cost():
    z = torch.matmul(x, w) + b
    cost_i = F.cross_entropy(z, y) # softmax 과정 포함
    c = torch.mean(cost_i)
    return c

In [34]:
# Training
optimizer = Adam([w, b], lr=0.01)

for epoch in range(2000):
    optimizer.zero_grad()
    c = cost()
    c.backward()
    optimizer.step()
    print(f'cost : {c.item()}')

cost : 1.551692247390747
cost : 1.465008020401001
cost : 1.3869930505752563
cost : 1.318434715270996
cost : 1.2598071098327637
cost : 1.2111135721206665
cost : 1.17177414894104
cost : 1.1406394243240356
cost : 1.1161808967590332
cost : 1.0968257188796997
cost : 1.0812772512435913
cost : 1.068674087524414
cost : 1.058560848236084
cost : 1.0507493019104004
cost : 1.045157551765442
cost : 1.0416772365570068
cost : 1.0400890111923218
cost : 1.0400302410125732
cost : 1.0410078763961792
cost : 1.042447566986084
cost : 1.043772578239441
cost : 1.0444841384887695
cost : 1.0442311763763428
cost : 1.0428438186645508
cost : 1.0403327941894531
cost : 1.0368585586547852
cost : 1.0326802730560303
cost : 1.0281000137329102
cost : 1.023410439491272
cost : 1.0188504457473755
cost : 1.0145827531814575
cost : 1.010683536529541
cost : 1.0071526765823364
cost : 1.0039383172988892
cost : 1.0009608268737793
cost : 0.998144268989563
cost : 0.9954323768615723
cost : 0.9927981495857239
cost : 0.9902429580688477

In [36]:
# 예측 함수 정의
def hxfn(xd):
    xd = torch.FloatTensor(xd)
    z = torch.matmul(xd, w) + b
    hx = torch.softmax(z, dim=1)
    return hx

In [41]:
# 예측하기 (공부시간 6, 출석횟수 6)
pred = hxfn([[6, 6]])
prob, idx = pred.max(dim=1) # row에서 가장 큰 확률값과 인덱스
print(prob, idx) # 0: A학점, 1: B학점, 2: C학점

tensor([0.6406], grad_fn=<MaxBackward0>) tensor([0])


In [54]:
# Quiz: (공부5, 출석6)인 경우 최종점수, (공부1, 출석2)인 경우 최종점수 예측하기
prob, idx = hxfn([[5, 6], [1, 2]]).max(dim=1)
print(f'{chr(idx.tolist()[0] + 65)}학점, {chr(idx.tolist()[1] + 65)}학점')

A학점, C학점


---

In [68]:
# High Level
model = Sequential()
model.add_module('nn', Linear(2, 3))
model.add_module('softmax', Softmax(dim=1)) # row 단위로
loss_fn = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.01)

for epoch in range(2000):
    optimizer.zero_grad()
    hx = model(x) # z = matmul(x, w) + b, hx = e^z / sum(e^z)
    cost = loss_fn(hx, y)
    cost.backward()
    optimizer.step()
    print(f'cost : {cost.item()}')

cost : 1.1702444553375244
cost : 1.1663105487823486
cost : 1.1623045206069946
cost : 1.158227562904358
cost : 1.1540801525115967
cost : 1.1498627662658691
cost : 1.1455742120742798
cost : 1.1412116289138794
cost : 1.1367697715759277
cost : 1.1322404146194458
cost : 1.1276130676269531
cost : 1.1228766441345215
cost : 1.1180245876312256
cost : 1.113063097000122
cost : 1.1080204248428345
cost : 1.10295569896698
cost : 1.0979690551757812
cost : 1.0932061672210693
cost : 1.088862657546997
cost : 1.0851682424545288
cost : 1.0823383331298828
cost : 1.0804680585861206
cost : 1.079406976699829
cost : 1.0787850618362427
cost : 1.0781954526901245
cost : 1.0773310661315918
cost : 1.0760302543640137
cost : 1.0742683410644531
cost : 1.0721209049224854
cost : 1.0697182416915894
cost : 1.0672051906585693
cost : 1.0647085905075073
cost : 1.0623178482055664
cost : 1.0600743293762207
cost : 1.0579743385314941
cost : 1.0559769868850708
cost : 1.0540180206298828
cost : 1.0520236492156982
cost : 1.049924492

In [69]:
# 학습된 w, b값 확인
list(model[0].parameters())

[Parameter containing:
 tensor([[ 2.6044,  0.5042],
         [-3.2602,  2.8020],
         [ 2.4510, -2.3393]], requires_grad=True),
 Parameter containing:
 tensor([-7.9743, -1.5333,  6.9967], requires_grad=True)]

In [74]:
# 예측하기
pred = model(torch.FloatTensor([[6, 7]]))
prob, idx = pred.max(dim=1)
print(f'6시간 공부하고 7회 출석한 경우 => {chr(idx.tolist()[0] + 65)}학점')

6시간 공부하고 7회 출석한 경우 => A학점
